In [1]:
import numpy as np
import  matplotlib.pyplot as plt

In [28]:
#Getting the data
data = np.genfromtxt('Half_kernel.txt', delimiter = ',')
np.random.shuffle(data)
x = data[:, (0,1)]
def normalize(X):
        mean=np.mean(X,axis=0)
        X=X-mean
        return X
normalize(x)
y = data[:, 2]
y = y.reshape(1000, 1)
m = len(x)
n = len(np.unique(y))
x = x.reshape(x.shape[1], m)
y = y.reshape(1, m)

In [29]:
def sigmoid(z):
    
    sigZ = 1/(1 + np.exp(-z))
    return sigZ

In [30]:
def layerSize(x, y, n):
    
    n_I = x.shape[0]
    n_h1 = 20          #Number of neurons in hidden layer 1 and 2.
    n_h2 = 15
    n_O = y.shape[0]
    
    return n_I, n_h1, n_h2, n_O
    
    

In [31]:
def Initialise(n_I, n_h1, n_h2, n_O):
    #Weights initialisation
    w1 = np.random.randn(n_h1, n_I)*0.01
    w2 = np.random.rand(n_h2, n_h1)*0.01
    w3 = np.random.randn(n_O, n_h2)*0.01
    b1 = np.zeros((n_h1, 1))
    b2 = np.zeros((n_h2, 1))
    b3 = np.zeros((n_O, 1))
    
    WandB = {"w1": w1, "w2": w2, "w3": w3,
                  "b1": b1,"b2": b2, "b3": b3}
    return WandB

In [32]:
def forward_prop(x, WandB):
    
    #Getting the weights and biases
    w1 = WandB["w1"]
    w2 = WandB["w2"]
    w3 = WandB["w3"]
    b1 = WandB["b1"]
    b2 = WandB["b2"]
    b3 = WandB["b3"]
    
    z1 = np.dot(w1, x) + b1
    a1 = np.tanh(z1)
    z2 = np.dot(w2, a1) + b2
    a2 = np.tanh(z2)
    z3 = np.dot(w3, a2) + b3
    a3 = sigmoid(z3)
    
    ZandA = {"z1": z1, "z2": z2, "z3": z3,
             "a1": a1, "a2": a2, "a3": a3}
    return ZandA

In [33]:
def cost_function(y, ZandA, m):
    
    a3 = ZandA["a3"]
    cost = (-1/m)*np.sum( y*np.log(a3) + (1-y)*np.log(1-a3))
    
    return np.squeeze(cost)

In [34]:
def backward_prop(x, y, m, WandB, ZandA):
    
    #Getting the required weights
    w1 = WandB["w1"]
    w2 = WandB["w2"]
    w3 = WandB["w3"]
    
    #Getting the required activations and z's
    a1 = ZandA["a1"]
    a2 = ZandA["a2"]
    a3 = ZandA["a3"]
    
    #Calculating the gradients 
    dz3 = a3 - y
    dw3 = np.dot(dz3, a2.T)/m
    db3 = np.sum(dz3, axis = 1, keepdims = True)/m
    dz2 = np.dot(w3.T, dz3) * (1 - a2**2)
    dw2 = np.dot(dz2, a1.T)/m
    db2 = np.sum(dz2, axis = 1, keepdims = True)/m
    dz1 = np.dot(w2.T, dz2) * (1 - a1**2)
    dw1 = np.dot(dz1, x.T)/m
    db1 = np.sum(dz1, axis = 1, keepdims = True)/m
    
    gradWandB = {"dw1": dw1, "dw2": dw2,"dw3": dw3,
             "db1": db1, "db2": db2, "db3": db3}
    
    return gradWandB

In [35]:
def updateWandB(WandB, gradWandB, alpha = 0.005):
    
    #Getting the weights and biases
    w1 = WandB["w1"]
    w2 = WandB["w2"]
    w3 = WandB["w3"]
    b1 = WandB["b1"]
    b2 = WandB["b2"]
    b3 = WandB["b3"]
    
    #Getting the gradients of weights and biases
    dw1 = gradWandB["dw1"]
    dw2 = gradWandB["dw2"]
    dw3 = gradWandB["dw3"]
    db1 = gradWandB["db1"]
    db2 = gradWandB["db2"]
    db3 = gradWandB["db3"]
       
    #Updating the parameters
    w1 -= alpha*dw1
    w2 -= alpha*dw2
    w3 -= alpha*dw3
    b1 -= alpha*db1
    b2 -= alpha*db2
    b3 -= alpha*db3
    
    updatedWandB = {"w1": w1, "w2": w2, "w3": w3,
                  "b1": b1,"b2": b2, "b3": b3}
    return updatedWandB

In [44]:
def neuralNetInAction(x, y, n_I, n_h1, n_h2, n_O, num_of_iter = 10000):
    
    #Getting the weights and biases
    WandB = Initialise(n_I, n_h1, n_h2, n_O)

    for i in range(0, num_of_iter):
         
        ZandA = forward_prop(x, WandB)
    
        cost = cost_function(y, ZandA, m)
        
        gradWandB = backward_prop(x, y, m, WandB, ZandA)
        
        WandB = updateWandB(WandB, gradWandB, alpha = 0.9)
        if (i % 5000 == 0):
            print ("Cost after iteration %i: %f" %(i, cost))
    
    return WandB

In [45]:
def predict(x, y, n,n_I, n_h1, n_h2, n_O):
    
    WandB = neuralNetInAction(x, y, n_I, n_h1, n_h2, n_O, num_of_iter = 50000)
    ZandA = forward_prop(x, WandB)
    a3 = ZandA["a3"]
    predictions = np.round(a3)
    
    return predictions

In [46]:
n_I, n_h1, n_h2, n_O = layerSize(x, y, n) 
y_pred = predict(x, y, n, n_I, n_h1, n_h2, n_O)

Cost after iteration 0: 0.693148
Cost after iteration 5000: 0.636036
Cost after iteration 10000: 0.602920
Cost after iteration 15000: 0.570878
Cost after iteration 20000: 0.571620
Cost after iteration 25000: 0.555629
Cost after iteration 30000: 0.568304
Cost after iteration 35000: 0.584363
Cost after iteration 40000: 0.540278
Cost after iteration 45000: 0.563580


In [47]:
a= (y_pred==y)
#         acc=np.mean(a)*100
#         return acc
np.mean(a) * 100

67.80000000000001

In [151]:
y_pred

array([[1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1.,
        0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0.,
        1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 1.,
        1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0.,
        0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1.,
        1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0.,
        1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1.,
        0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1.,
        0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0.,
        0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0.,
        0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1.,
        0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0.,
        1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0.,
        1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 

In [152]:
y

array([[0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1.,
        0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1.,
        1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1.,
        0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
        1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1.,
        1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1.,
        0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0.,
        0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0.,
        1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0.,
        0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0.,
        1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1.,
        0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 